In [104]:
from coffea.nanoevents import NanoEventsFactory, FCC, BaseSchema

events = NanoEventsFactory.from_root(
    "../../../../data/eos/experiment/fcc/ee/generation/DelphesEvents/winter2023/IDEA/p8_ee_ZZ_ecm240/events_136205668.root:events",
    schemaclass=FCC.get_schema("pre-edm4hep1"),
    # schemaclass=BaseSchema,
    entry_stop=10000,
    delayed=False,
    uproot_options = {"filter_name": lambda x : "PARAMETERS" not in x}
).events()

/home/prayag/coffeafcc/development/coffea/src/coffea/nanoevents/mapping/uproot.py:153: UserWarning: Skipping PARAMETERS/_intMap/_intMap.first as it is not interpretable by Uproot
  warnings.warn(f"Skipping {key} as it is not interpretable by Uproot")
/home/prayag/coffeafcc/development/coffea/src/coffea/nanoevents/mapping/uproot.py:153: UserWarning: Skipping PARAMETERS/_intMap/_intMap.second as it is not interpretable by Uproot
  warnings.warn(f"Skipping {key} as it is not interpretable by Uproot")
/home/prayag/coffeafcc/development/coffea/src/coffea/nanoevents/mapping/uproot.py:153: UserWarning: Skipping PARAMETERS/_floatMap/_floatMap.first as it is not interpretable by Uproot
  warnings.warn(f"Skipping {key} as it is not interpretable by Uproot")
/home/prayag/coffeafcc/development/coffea/src/coffea/nanoevents/mapping/uproot.py:153: UserWarning: Skipping PARAMETERS/_floatMap/_floatMap.second as it is not interpretable by Uproot
  warnings.warn(f"Skipping {key} as it is not interpretabl

In [105]:
# Helper functions
import awkward as ak
import numpy as np
import vector
vector.register_awkward()

In [106]:
# 1. To replace: FCCAnalyses::ZHfunctions::resonanceBuilder_mass(91.2,false)(selected_muons, MCRecoAssociations0, MCRecoAssociations1, ReconstructedParticles, Particle, Particle0, Particle1)
def resonanceBuilder_mass(resonance_mass=None, use_MC_Kinematics=False, leptons=None, MCRecoAssociations=None, ReconstructedParticles=None, MCParticles=None):
    '''
    Build the Z resonance based on the available leptons. Returns the best lepton pair compatible with the Z mass and recoil at 125 GeV
    technically, it returns a ReconstructedParticleData object with index 0 the di-lepton system, index and 2 the leptons of the pair
    '''
    if leptons is None:
        raise AttributeError("No leptons passed")
    #Create all the combinations
    combs = ak.combinations(leptons,2)
    # Get dileptons
    lep1 , lep2 = ak.unzip(combs)
    di_lep = lep1 + lep2 # This process drops any other field except 4 momentum fields

    di_lep["charge"] =  lep1.charge + lep2.charge
    di_lep["l1_index"] = lep1.index
    di_lep["l2_index"] = lep2.index

    # Choose oppositely charged leptons
    di_lep = di_lep[di_lep.charge == 0]

    # Sort by closest mass to the resonance value
    sort_mask = ak.argsort(abs(resonance_mass-di_lep.mass), axis=1)
    Reso = di_lep[sort_mask]

    return Reso

In [107]:
# 2. To replace: FCCAnalyses::ZHfunctions::getTwoHighestPMuons(rest_of_muons)") # Find the higest p muon pair from the remaining muons (off-shell Z)
def getTwoHighestPMuons(muons):
    '''
    Sort by decending P and return the pair of particles with highest P and opposite charges
    '''
    # if not ak.all(ak.num(muons, axis=1) > 1 ):
    #     raise IndexError("Need at least two particles!")
    sorted_muons_p = ak.argsort(muons.p, ascending=False)
    sorted_muons = muons[sorted_muons_p]

    # First particle is always selected, if the second one has the opposite charge, then its accepted otherwise we move on to the third and so on
    # Interestingly, this type of operation is non trivial in an array format
    first_muon, other_muons = sorted_muons[:, 0:1], sorted_muons[:, 1:]

    # prepare before cartesian : replace none with []
    first_muon = ak.fill_none(first_muon, [], axis=0)
    other_muons = ak.fill_none(other_muons, [], axis=0)
    # All combinations
    all_comb = ak.cartesian([first_muon, other_muons])
    l1, l2 = ak.unzip(all_comb)
    charge_mask = l1.charge!= l2.charge
    l1 = l1[charge_mask]
    l2 = l2[charge_mask]

    at_least_one_opp_charged = ak.sum(charge_mask, axis=1) > 0
    
    return ak.firsts(l1), ak.firsts(l2), at_least_one_opp_charged

In [108]:
# Main calculations
Muons = events.ReconstructedParticles[events.Muonidx0.index]
Muons["index"] = events.Muonidx0.index # Attach the reco index for easier calculations later

In [109]:
sel_muon = Muons.p > 2.0
selected_muons_p = Muons[sel_muon]

In [110]:
at_least_4_muons = ak.num(selected_muons_p, axis=1) > 3
selected_muons = ak.mask(selected_muons_p, at_least_4_muons)

In [111]:
# Build Z resonances
Z = resonanceBuilder_mass(resonance_mass=91.2, use_MC_Kinematics=False, leptons=selected_muons)

In [112]:
# On Shell Z
zll = ak.firsts(Z)

In [113]:
l1 = ak.firsts(selected_muons[selected_muons.index == zll.l1_index])
l2 = ak.firsts(selected_muons[selected_muons.index == zll.l2_index])

In [114]:
def create_mask(a, b, c):
    mask1 = a != c
    mask2 = b != c
    mask = mask1 & mask2
    return mask

In [115]:
mask = create_mask(zll.l1_index, zll.l2_index, selected_muons.index)

In [116]:
rest_of_muons = selected_muons[mask]

In [117]:
m1, m2, c_mask = getTwoHighestPMuons(rest_of_muons)

In [130]:
non_res_Z = m1 + m2
# Angle between the two
non_res_Z_angle = m1.deltaangle(m2)

In [132]:
# Collect all the four Muons
fourMuons_collected = ak.concatenate(
    (
        ak.mask(l1,c_mask)[:, np.newaxis],
        ak.mask(l2,c_mask)[:, np.newaxis],
        m1[:, np.newaxis],
        m2[:, np.newaxis]
    ),
    axis=1
)

In [133]:
fourMuons_p = fourMuons_collected.p

In [134]:
fourMuons_p

<Array [[None, None, None, None], ..., [...]] type='10000 * 4 * ?float32[pa...'>

In [135]:
fourMuons_collected = ak.mask(fourMuons_collected, ak.num(fourMuons_collected, axis=1) > 3)
fourMuons = ak.mask(zll, c_mask) + non_res_Z

In [136]:
non_res_Z_m = non_res_Z.m

In [137]:
zll_m = zll[c_mask].m

In [138]:
fourMuons_m = fourMuons.m

In [139]:
intLumi        = 10.80e+06 #in pb-1
scale = 52.6539*intLumi/100000
import hist

In [140]:
res_Z_hist = hist.Hist.new.Regular(50,0,250).Double().fill(ak.drop_none(zll_m))*scale
non_res_Z_hist = hist.Hist.new.Regular(50,0,250).Double().fill(ak.drop_none(non_res_Z_m))*scale
four_muon_mass_hist = hist.Hist.new.Regular(50,0,250).Double().fill(ak.drop_none(fourMuons_m))*scale

In [141]:
res_Z_hist

Hist(Regular(50, 0, 250, label='Axis 0'), storage=Double()) # Sum: 142165.53

In [142]:
non_res_Z_hist

Hist(Regular(50, 0, 250, label='Axis 0'), storage=Double()) # Sum: 142165.53

In [143]:
four_muon_mass_hist

Hist(Regular(50, 0, 250, label='Axis 0'), storage=Double()) # Sum: 142165.53

In [144]:

def remove(array, idx):
    '''
    Returns all the particles except the indices defined in idx.index.
    Eg. remove(events.ReconstructedParticle, events.Muonidx0) returns the events.ReconstructedParticle array with all the muons removed
    '''
    index = idx.index
    all_index = ak.local_index(array,axis=1)

    i,a = ak.unzip(ak.cartesian([index[:,np.newaxis] ,all_index], nested=True))
    c = a == i
    d = ak.firsts(c)
    s = ak.sum(d, axis=2)
    kl = s == 1
    
    return array[~kl]

In [153]:
# 3. To sum all the lorentz vectors in a an array of lorentzvectors
def sum_all(array_of_lv):
    # array_of_lv = ak.drop_none(array_of_lv)

    out = ak.zip(
        {
            "px":ak.sum(array_of_lv.px , axis=1),
            "py":ak.sum(array_of_lv.py , axis=1),
            "pz":ak.sum(array_of_lv.pz , axis=1),
            "E":ak.sum(array_of_lv.E , axis=1)
        },
        with_name="Momemtum4D"
    )

    return out
def recoilBuilder(vec, ecm):
    '''
    Builds Recoil from a given LorentzVector and Center of Mass Energy
    Input:    vec(var*[var*LorentzVector]),
              ecm(float)
    Output: Recoil([var*LorentzVecctor])
    '''
    Recoil = ak.zip({"px":0.0-vec.px,"py":0.0-vec.py,"pz":0.0-vec.pz,"E":ecm-vec.E},with_name="Momentum4D")
    return Recoil

# 4. To replace : FCCAnalyses::ZHfunctions::coneIsolation(0.0,0.523599)(fourMuons,rest_of_particles)
def coneIsolation(particle, rest_of_the_particles, min_dr=0.0 , max_dr=0.4):
    ''' Refer: https://github.com/delphes/delphes/blob/master/modules/Isolation.cc#L154
    '''
    neutral_particles = ak.mask(rest_of_the_particles, rest_of_the_particles.charge == 0)
    charged_particles = ak.mask(rest_of_the_particles, rest_of_the_particles.charge != 0)

    n_combs = ak.cartesian((particle,neutral_particles[:,np.newaxis]), axis=1)
    n1,n2 = ak.unzip(n_combs)
    c_combs = ak.cartesian((particle,charged_particles[:,np.newaxis]), axis=1)
    c1,c2 = ak.unzip(c_combs)

    n_angle = n1.deltaangle(n2)
    c_angle = c1.deltaangle(c2)

    n_angle_mask = (n_angle < max_dr) & (n_angle > min_dr)
    c_angle_mask = (c_angle < max_dr) & (c_angle > min_dr)

    filtered_neutral = n2[n_angle_mask]
    filtered_charged = c2[c_angle_mask]

    sumNeutral = ak.sum(filtered_neutral.p, axis=2)
    sumCharged = ak.sum(filtered_charged.p, axis=2)

    total_sum = sumNeutral + sumNeutral

    ratio = total_sum / particle.p

    return ratio


In [154]:
fourMuons_pmin = ak.min(fourMuons_collected.p, axis=1)

chosen_reco_4_mu = ak.mask(events.ReconstructedParticles, at_least_4_muons)
chosen_reco = ak.mask(chosen_reco_4_mu,c_mask)

rest_of_particles = remove(chosen_reco, fourMuons_collected)
all_others = sum_all(rest_of_particles)

Emiss = recoilBuilder(sum_all(chosen_reco), ecm=240)
pmiss = Emiss.E

# Cone Isolation
fourMuons_iso = coneIsolation(fourMuons_collected, rest_of_particles, min_dr=0.0, max_dr=0.523599)
fourMuons_min_iso = ak.max(fourMuons_iso, axis=1)


# fourMuons_pmin = ak.min(fourMuons_collected.p, axis=1)

# chosen_reco_4_mu = events.ReconstructedParticles[at_least_4_muons]
# chosen_reco = chosen_reco_4_mu[c_mask]

# rest_of_particles = remove(chosen_reco, fourMuons_collected)
# all_others = functions.sum_all(rest_of_particles)

# Emiss = recoilBuilder(functions.sum_all(chosen_reco), ecm=config.ecm)
# pmiss = Emiss.E

# # Cone Isolation
# fourMuons_iso = functions.coneIsolation(fourMuons_collected, rest_of_particles, min_dr=0.0, max_dr=0.523599)
# fourMuons_min_iso = ak.max(fourMuons_iso, axis=1)

In [155]:
pmiss

<Array [None, None, None, None, ..., None, None, None] type='10000 * ?float32'>

In [149]:
fourMuons_iso

<Array [[None, None, None, None], ..., [...]] type='10000 * option[var * ?f...'>

In [150]:
fourMuons_pmin

<Array [None, None, None, None, ..., None, None, None] type='10000 * ?float32'>

In [151]:
four_muon_mass_hist

Hist(Regular(50, 0, 250, label='Axis 0'), storage=Double()) # Sum: 142165.53